In [ ]:
import os
from itertools import product
from astropy.io import fits
from astropy.table import Table
from lvmdrp.utils import metadata as md
from lvmdrp.functions import run_drp as drp
from lvmdrp.functions import imageMethod as image_tasks
from lvmdrp.functions import rssMethod as rss_tasks
from lvmdrp.core.constants import SPEC_CHANNELS

# Instructions

This notebook implementes a quick reduction of the LVM data. Main asumptions are:

1. There is a `data_calib` directory containing master calibration frames for:
    - pixel mask (`lvm-mpixmask-{CAMERA}.fits`)
    - bias (`lvm-mbias-{CAMERA}.fits`)
    - dark (`lvm-mdark-{CAMERA}.fits`)
    - traces (`traces_{CAMERA}_p4.fits`)
    - wavelengths (`lvm-mwave_neon-{CAMERA}.fits`)
    - LSF (`lvm-mlsf_neon-{CAMERA}.fits`)

2. There is a `data_science` directory containing the **raw** target science exposures (`sdR-s-{CAMERA}-{EXPNUM:>08d}.fits.gz`)

3. Data products from this reduction can be stored in a directory `data_products` (this directory will be created by this notebook if it does not exists).

This reduction will push raw frames from preprocessing down to wavelength calibration.

In [ ]:
# define paths
CALDIR = "data_calib/"
SCIDIR = "data_science/"
OUTDIR = "data_products/"
os.makedirs(OUTDIR, exist_ok=True)

# define cameras
CAMERAS = [f"{channel}{spec}" for channel, spec in product(("brz"), (1,2,3))]

In [ ]:
# target science directory
sci_paths = sorted([os.path.join(SCIDIR, sci_name) for sci_name in os.listdir(SCIDIR)])
print(sci_paths)

In [ ]:
for isci, sci_path in enumerate(sci_paths):
    # get basic metadata
    sci_header = fits.getheader(sci_path, ext=0)
    sci_camera = sci_header["CCD"]
    sci_exposure = sci_header["EXPOSURE"]
    
    # define calibration frames paths
    mpixmask_path = os.path.join(CALDIR, f"lvm-mpixmask-{sci_camera}.fits")
    mbias_path = os.path.join(CALDIR, f"lvm-mbias-{sci_camera}.fits")
    mdark_path = os.path.join(CALDIR, f"lvm-mdark-{sci_camera}.fits")
    mtrace_path = os.path.join(CALDIR, f"traces_{sci_camera}_p4.fits")
    mwave_path = os.path.join(CALDIR, f"lvm-mwave_neon-{sci_camera}.fits")
    mlsf_path = os.path.join(CALDIR, f"lvm-mlsf_neon-{sci_camera}.fits")
    
    # define products paths
    psci_path = os.path.join(OUTDIR, f"lvm-pobject-{sci_camera}-{sci_exposure:>08d}.fits")
    dsci_path = os.path.join(OUTDIR, f"lvm-dobject-{sci_camera}-{sci_exposure:>08d}.fits")
    xsci_path = os.path.join(OUTDIR, f"lvm-xobject-{sci_camera}-{sci_exposure:>08d}.fits")
    wsci_path = os.path.join(OUTDIR, f"lvm-wobject-{sci_camera}-{sci_exposure:>08d}.fits")
    
    if os.path.isfile(wsci_path):
        print(f"skipping {wsci_path}, file already exist")
        continue
    
    # preprocess frame
    image_tasks.preproc_raw_frame(in_image=sci_path, out_image=psci_path, in_mask=mpixmask_path)
    
    # detrend frame
    image_tasks.detrend_frame(in_image=psci_path, out_image=dsci_path, in_bias=mbias_path, in_dark=mdark_path, in_slitmap=Table(drp.fibermap.data))
    
    # extract 1d spectra
    image_tasks.extract_spectra(in_image=dsci_path, out_rss=xsci_path, in_trace=mtrace_path, method="aperture", aperture=3)
    
    # wavelength calibrate & resample
    iwave, fwave = SPEC_CHANNELS[sci_camera[0]]
    rss_tasks.create_pixel_table(in_rss=xsci_path, out_rss=wsci_path, arc_wave=mwave_path, arc_fwhm=mlsf_path)
    rss_tasks.resample_wavelength(in_rss=wsci_path, out_rss=wsci_path, method="linear", disp_pix=0.5, start_wave=iwave, end_wave=fwave, err_sim=10, parallel=0)
